In [1]:
import os
import sys

sys.path.append("../../../../../")
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [2]:
import copy
import torch
from datetime import datetime
from src.utils.helper import Config, color_print
from src.utils.load import load_model, load_data, save_checkpoint
from src.models.evaluate import (
    evaluate_model,
    get_sparsity,
    get_similarity,
    get_perplexity,
)
from src.utils.sampling import SamplingDataset
from src.pruning.prune_head import head_importance_prunning
from src.utils.helper import report_to_df, append_nth_row

In [3]:
name = "OSDG"
device = torch.device("cuda:0")
checkpoint = None
batch_size = 16
num_workers = 4
num_samples = 16
ratio = 0.3
seed = 44

In [4]:
script_start_time = datetime.now()
print(f"Script started at: {script_start_time.strftime('%Y-%m-%d %H:%M:%S')}")

Script started at: 2024-10-23 21:26:30


In [5]:
config = Config(name, device)
num_labels = config.config["num_labels"]
model = load_model(config)

Loading the model.

{

'architectures'

: 

'bert'

...

'tokenizer_name'

: 

'sadickam/sdg-classification-bert'

}

The model sadickam/sdg-classification-bert is loaded.

In [6]:
train_dataloader, valid_dataloader, test_dataloader = load_data(
    config,
    batch_size=batch_size,
    num_workers=num_workers,
    do_cache=True,
)

Loading cached dataset OSDG.

train.pkl is loaded from cache.

valid.pkl is loaded from cache.

test.pkl is loaded from cache.

The dataset OSDG is loaded

...

,
 

'path'

: 

'albertmartinez/OSDG'

}

In [7]:
# print("Evaluate the original model")
# result = evaluate_model(model, config, test_dataloader)

In [8]:
config.init_seed()
all_samples = SamplingDataset(
    train_dataloader,
    config,
    200,
    num_samples,
    False,
    4,
    resample=False,
)

In [9]:
result_list = []

module = copy.deepcopy(model)

head_importance_prunning(module, config, all_samples, ratio)

print(f"Evaluate the pruned model")
result = evaluate_model(module, config, test_dataloader, verbose=True)
result_list.append(result)

Total heads to prune: 43

tensor([[0.4747, 0.4317, 0.4891, 0.5093, 0.4567, 0.4666, 0.5067, 0.4785, 0.5582,
         0.4236, 0.5825, 0.4175],
        [0.4234, 0.6757, 0.3818, 0.3243, 0.6439, 0.4396, 0.6521, 0.3851, 0.3816,
         0.3963, 0.4775, 0.5047],
        [0.6747, 0.4075, 0.2934, 0.2980, 0.3167, 0.3683, 0.3579, 0.3163, 0.2893,
         0.7107, 0.3948, 0.3906],
        [0.4759, 0.4354, 0.3277, 0.4589, 0.3294, 0.6723, 0.3433, 0.3908, 0.4428,
         0.6593, 0.3811, 0.6148],
        [0.7708, 0.3074, 0.3783, 0.5660, 0.4790, 0.4956, 0.3807, 0.3298, 0.2292,
         0.4202, 0.5358, 0.5003],
        [0.4186, 0.3720, 0.3554, 0.5116, 0.4853, 0.2781, 0.3575, 0.5138, 0.3230,
         0.7219, 0.6544, 0.2794],
        [0.4631, 0.4400, 0.4843, 0.4607, 0.4504, 0.6346, 0.6040, 0.4176, 0.4031,
         0.6338, 0.3654, 0.5141],
        [0.6227, 0.5131, 0.4976, 0.3439, 0.6072, 0.5023, 0.3705, 0.5096, 0.4806,
         0.6561, 0.5942, 0.4583],
        [0.5007, 0.4401, 0.5599, 0.5067, 0.4545, 0.5377, 0.5388, 0.3886, 0.3313,

{(3, 4), (5, 1), (11, 2), (10, 0), (8, 9), (11, 5), (2, 2), (10, 3), (9, 11), (11, 8), (2, 5), (1, 3), (2, 8), (4, 2), (5, 6), (4, 8), (3, 6), (11, 4), (10, 2), (9, 4), (11, 1), (8, 8), (2, 4), (10, 11), (11, 10), (10, 8), (2, 7), (7, 3), (7, 6), (6, 10), (3, 2), (4, 1), (4, 7), (5, 2), (5, 5), (5, 11), (5, 8), (11, 3), (11, 9), (10, 7), (11, 6), (2, 3), (2, 6)}

Evaluate the pruned model

Evaluating the model:   0%|                                                                                   …

In [10]:
for concern in range(num_labels):
    config.init_seed()
    get_similarity(model, module, valid_dataloader, concern, num_samples, config)

adding eps to diagonal and taking inverse

taking square root

dot products...

trying to take final svd

computed everything!

...

CCA coefficients mean non-concern: 0.8429712921253724

Linear CKA concern: 0.7402190575426008

Linear CKA non-concern: 0.889669220287899

Kernel CKA concern: 0.7108695416485651

Kernel CKA non-concern: 0.8854514256148078

In [11]:
get_sparsity(module)
print("original model's perplexity")
get_perplexity(model, valid_dataloader, config)
print("pruned model's perplexity")
get_perplexity(module, valid_dataloader, config)

0.09873972232069293

{'bert.encoder.layer.0.attention.self.query.weight': 0.0, 'bert.encoder.layer.0.attention.self.query.bias': 0.0, 'bert.encoder.layer.0.attention.self.key.weight': 0.0, 'bert.encoder.layer.0.attention.self.key.bias': 0.0, 'bert.encoder.layer.0.attention.self.value.weight': 0.0, 'bert.encoder.layer.0.attention.self.value.bias': 0.0, 'bert.encoder.layer.0.attention.output.dense.weight': 0.0, 'bert.encoder.layer.0.attention.output.dense.bias': 0.0, 'bert.encoder.layer.0.intermediate.dense.weight': 0.0, 'bert.encoder.layer.0.intermediate.dense.bias': 0.0, 'bert.encoder.layer.0.output.dense.weight': 0.0, 'bert.encoder.layer.0.output.dense.bias': 0.0, 'bert.encoder.layer.1.attention.self.query.weight': 0.08333333333333333, 'bert.encoder.layer.1.attention.self.query.bias': 0.0, 'bert.encoder.layer.1.attention.self.key.weight': 0.08333333333333333, 'bert.encoder.layer.1.attention.self.key.bias': 0.0, 'bert.encoder.layer.1.attention.self.value.weight': 0.08333333333333333, 'bert.encoder.layer.1.

original model's perplexity

2.445301055908203

pruned model's perplexity

2.3936240673065186

2.3936240673065186

In [12]:
df_list = [report_to_df(df) for df in result_list]
csv_name = f"{datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}"
new_df = df_list[0]
new_df.to_csv(f"results/{csv_name}.csv", index=False)
print(csv_name)
new_df

2024-10-23_21-32-30

,class,precision,recall,f1-score,support
0,0,0.7587,0.6587,0.7052,797
1,1,0.8368,0.7213,0.7748,775
2,2,0.8532,0.8843,0.8684,795
3,3,0.8802,0.8072,0.8421,1110
4,4,0.8468,0.7984,0.8219,1260
5,5,0.8897,0.6859,0.7746,882
6,6,0.8663,0.7723,0.8166,940
7,7,0.4799,0.5814,0.5258,473
8,8,0.6649,0.8485,0.7456,746
9,9,0.6046,0.6923,0.6455,689
